In [ ]:
from transformers import pipeline, TFAutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from transformers import Trainer, TrainingArguments, DefaultDataCollator
from datasets import Dataset
from typing import Dict, List, Optional
import tensorflow as tf
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BrandSentimentAnalyzer:
    def __init__(self):
        """Initialize all transformer models with TensorFlow backend"""
        # Emotion detection model
        self.emotion_model = pipeline(
            'text-classification',
            model='SamLowe/roberta-base-go_emotions',
            return_all_scores=True,
            framework="tf"
        )
        
        # Sarcasm detection model
        self.sarcasm_tokenizer = AutoTokenizer.from_pretrained("MohamedGalal/marbert-sarcasm-detector")
        self.sarcasm_model = TFAutoModelForSequenceClassification.from_pretrained(
            "MohamedGalal/marbert-sarcasm-detector"
        )

        # Sentiment analysis model
        self.sentiment_model = pipeline(
            "sentiment-analysis",
            model="cardiffnlp/twitter-roberta-base-sentiment-latest",
            framework="tf"
        )

        # Brand perception model
        self.perception_model = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli",
            framework="tf"
        )
        
        # Configure device (GPU if available)
        self.device = "cuda" if tf.config.list_physical_devices('GPU') else "cpu"
        logger.info(f"Using device: {self.device}")

    def _preprocess_emotion_data(self, texts: List[str], labels: List[Dict[str, float]]) -> Dataset:
        """Convert emotion data to HuggingFace Dataset format"""
        data = {"text": texts, "labels": labels}
        return Dataset.from_dict(data)

    def fine_tune_emotion_model(self, train_texts: List[str], train_labels: List[Dict[str, float]], 
                               val_texts: Optional[List[str]] = None, val_labels: Optional[List[Dict[str, float]]] = None,
                               epochs: int = 3, batch_size: int = 8):
        """
        Fine-tune the emotion detection model
        
        Args:
            train_texts: List of training texts
            train_labels: List of dictionaries with emotion scores
            val_texts: Optional validation texts
            val_labels: Optional validation labels
            epochs: Number of training epochs
            batch_size: Batch size for training
        """
        logger.info("Fine-tuning emotion model...")
        
        # Convert labels to the format expected by the model
        emotion_labels = list(train_labels[0].keys())
        label2id = {label: idx for idx, label in enumerate(emotion_labels)}
        
        # Convert training data
        train_dataset = self._preprocess_emotion_data(train_texts, train_labels)
        
        # Tokenize data
        tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
        
        def tokenize_function(examples):
            return tokenizer(examples["text"], padding="max_length", truncation=True)
        
        tokenized_train = train_dataset.map(tokenize_function, batched=True)
        
        # Prepare model for fine-tuning
        config = AutoConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
        model = TFAutoModelForSequenceClassification.from_pretrained(
            "SamLowe/roberta-base-go_emotions",
            config=config
        )
        
        # Training setup would go here (simplified for example)
        # In practice, you'd need to implement proper training loop
        logger.info("Emotion model fine-tuning setup complete (implementation needed)")

    def detect_emotions(self, text: str) -> Dict:
        """Detect emotions in text with confidence scores"""
        results = self.emotion_model(text)[0]
        return {item['label']: item['score'] for item in results}
    
    def detect_sarcasm(self, text: str) -> Dict:
        """Detect sarcasm with confidence scores"""
        inputs = self.sarcasm_tokenizer(text, return_tensors="tf", truncation=True)
        outputs = self.sarcasm_model(**inputs)
        probs = tf.nn.softmax(outputs.logits, axis=1)
        return {
            'sarcastic': float(probs[0][1]),
            'literal': float(probs[0][0])
        }
    
    def analyze_sentiment(self, text: str) -> Dict:
        """Analyze sentiment with confidence score"""
        result = self.sentiment_model(text)[0]
        return {'label': result['label'], 'score': result['score']}
    
    def assess_brand_perception(self, text: str, brand_name: str) -> Dict:
        """Assess brand perception across multiple dimensions"""
        candidate_labels = [
            "positive", "negative", "trustworthy", 
            "innovative", "expensive", "good value",
            "reliable", "poor quality", "luxury",
            "outdated", "trendy", "ethical"
        ]
        result = self.perception_model(text, candidate_labels, multi_label=True)
        return {
            'brand': brand_name,
            'perception_scores': dict(zip(result['labels'], result['scores']))
        }
    
    def calculate_composite_sentiment(self, analysis: Dict) -> float:
        """
        Calculate composite sentiment score (0-1 scale) combining:
        - Emotion analysis
        - Sarcasm detection
        - Sentiment analysis
        - Brand perception
        """
        # Positive emotions contribute positively
        positive_emotions = ['admiration', 'amusement', 'approval', 'caring', 'desire',
                            'excitement', 'gratitude', 'joy', 'love', 'optimism', 'pride', 'relief']
        emotion_score = sum(analysis['emotions'].get(emotion, 0) for emotion in positive_emotions)
        
        # Negative emotions contribute negatively
        negative_emotions = ['anger', 'annoyance', 'disappointment', 'disapproval', 
                           'disgust', 'embarrassment', 'fear', 'grief', 'nervousness',
                           'remorse', 'sadness']
        emotion_score -= sum(analysis['emotions'].get(emotion, 0) for emotion in negative_emotions)
        
        # Normalize emotion score (-1 to 1 range)
        emotion_score = emotion_score / len(analysis['emotions'])
        
        # Adjust for sarcasm (sarcastic comments often flip sentiment)
        sarcasm_adjustment = 1 - (2 * analysis['sarcasm']['sarcastic'])
        
        # Get sentiment score (-1 for negative, 0 for neutral, 1 for positive)
        sentiment_map = {'negative': -1, 'neutral': 0, 'positive': 1}
        sentiment_score = sentiment_map.get(analysis['sentiment']['label'].lower(), 0) * analysis['sentiment']['score']
        
        # Get brand perception adjustment
        perception_score = (
            analysis['brand_perception']['perception_scores'].get('positive', 0) -
            analysis['brand_perception']['perception_scores'].get('negative', 0)
        )
        
        # Combine scores with weights
        composite_score = (
            0.4 * emotion_score + 
            0.3 * sentiment_score * sarcasm_adjustment + 
            0.3 * perception_score
        )
        
        # Normalize to 0-1 range
        return (composite_score + 1) / 2
    
    def generate_perception_wordcloud(self, perception_scores: Dict[str, float], brand_name: str):
        """
        Generate a word cloud visualization of brand perception
        
        Args:
            perception_scores: Dictionary of perception scores from assess_brand_perception
            brand_name: Brand name for title
            
        Returns:
            matplotlib Figure object
        """
        # Create word frequencies dictionary
        word_freq = {k: v * 100 for k, v in perception_scores.items()}
        
        # Generate word cloud
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
        
        # Plot
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.set_title(f"Brand Perception: {brand_name}", fontsize=15)
        ax.axis('off')
        
        return fig
    
    def full_analysis(self, text: str, brand_name: str, generate_visualization: bool = True) -> Dict:
        """
        Perform complete sentiment and brand analysis
        
        Args:
            text: Text to analyze
            brand_name: Brand name for perception analysis
            generate_visualization: Whether to generate word cloud
            
        Returns:
            Dictionary with complete analysis results
        """
        analysis = {
            "text": text,
            "emotions": self.detect_emotions(text),
            "sarcasm": self.detect_sarcasm(text),
            "sentiment": self.analyze_sentiment(text),
            "brand_perception": self.assess_brand_perception(text, brand_name)
        }
        
        # Calculate composite score
        analysis["composite_sentiment_score"] = self.calculate_composite_sentiment(analysis)
        
        # Generate visualization if requested
        if generate_visualization:
            analysis["perception_wordcloud"] = self.generate_perception_wordcloud(
                analysis["brand_perception"]["perception_scores"],
                brand_name
            )
        
        return analysis

# Example usage
if __name__ == "__main__":
    analyzer = BrandSentimentAnalyzer()
    
    # Example text and brand
    sample_text = "I love how this brand always delivers quality products, said no one ever!"
    brand = "ExampleCorp"
    
    # Perform full analysis
    results = analyzer.full_analysis(sample_text, brand)
    
    # Print results
    print(f"Composite Sentiment Score: {results['composite_sentiment_score']:.2f}")
    print("\nBrand Perception Scores:")
    for label, score in results["brand_perception"]["perception_scores"].items():
        print(f"{label}: {score:.2f}")
    
    # Show word cloud
    if "perception_wordcloud" in results:
        results["perception_wordcloud"].show()